In [108]:
!pip install langchain_community langchainhub chromadb langchain langchain-openai

In [109]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('final')


In [110]:
#step - documentation loader means raw dta text generate ho jaega full web page ka
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(["https://www.primevideo.com/"])
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://www.primevideo.com/', 'title': 'Prime Video: Watch movies, TV shows, sports, and live TV', 'description': 'Stream popular movies, TV shows, sports, and live TV included with Prime, and even more with subscriptions. Watch anywhere, anytime.', 'language': 'en-us'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPrime Video: Watch movies, TV shows, sports, and live TV\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMenuCaret DownBrowseHomeMoviesTV showsSportsLive TVSubscriptionsChannelsBrowse allCaret RightHomeMoviesTV showsSportsLive TVChannelsSubscriptionsSearchClearENCaret DownBahasa IndonesiaBahasa MelayuDanskDeutschEnglishEspañolEspañol LatinoaméricaFrançaisItalianoMagyarNederlandsNorskPolskiPortuguês (Brasil)Português (Portugal)RomânăSuomiSvenskaTürkçeWikang FilipinoČeštinaΕλληνικάРусскийעבריתالعربيةहिन्दीதமிழ்తెలుగుไทย日本語简体中文

In [111]:
#step - raw text data cleaning and chunking to make more semantic and useful short data rather than whole ramayana so llm mien tokens kam jaye embedding kam bane and transformer operation minimum karega saved tiome and money
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200  # chunk overlap (characters)
     # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(all_splits)
print(len(all_splits))

[Document(metadata={'source': 'https://www.primevideo.com/', 'title': 'Prime Video: Watch movies, TV shows, sports, and live TV', 'description': 'Stream popular movies, TV shows, sports, and live TV included with Prime, and even more with subscriptions. Watch anywhere, anytime.', 'language': 'en-us'}, page_content='Prime Video: Watch movies, TV shows, sports, and live TV'), Document(metadata={'source': 'https://www.primevideo.com/', 'title': 'Prime Video: Watch movies, TV shows, sports, and live TV', 'description': 'Stream popular movies, TV shows, sports, and live TV included with Prime, and even more with subscriptions. Watch anywhere, anytime.', 'language': 'en-us'}, page_content='MenuCaret DownBrowseHomeMoviesTV showsSportsLive TVSubscriptionsChannelsBrowse allCaret RightHomeMoviesTV showsSportsLive TVChannelsSubscriptionsSearchClearENCaret DownBahasa IndonesiaBahasa MelayuDanskDeutschEnglishEspañolEspañol LatinoaméricaFrançaisItalianoMagyarNederlandsNorskPolskiPortuguês (Brasil)Po

In [112]:
!pip install sentence-transformers
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
#issi steop mien hugging fdace se converetd text to embedding chroma db vector db mien store ho gyi...
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=embedding
)



In [113]:
#store in ram for sat access but temporary storage hoti not a mandatory step for rag
from langchain_community.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embedding)
vector_store.add_documents(all_splits)


['2e2b8955-320f-45d1-8ae6-6b6a40093682',
 'd7d911ac-0d0c-49d3-9afb-690196c36aaf',
 '44dfaf4b-358b-4f9b-a286-da0b872d4ea4']

In [114]:
#Rag pipeline retriever and augmentation
retriever=vectorstore.as_retriever()


In [115]:
#augmentation llm ko kya prompt bhejni along with question and contextual background from chroma db...
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:"""
)


In [116]:
!pip install -U langchain-community langchain-huggingface
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface import HuggingFaceEndpoint


from google.colab import userdata
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get("HUGGINGFACEHUB_API_TOKEN")


llm = ChatHuggingFace(
    llm=HuggingFaceEndpoint(
        repo_id="mistralai/Mistral-7B-Instruct-v0.2",
        task="conversational",
        max_new_tokens=256,
        temperature=0
    )
)


In [117]:
#when we want to pass questions directly as input we always use concept of runnable pass through
from langchain_core.runnables import RunnablePassthrough
#for conevrting final llm se mile ans to req ans we want only string part not bhaang bhosda so use stroutputparser
from langchain_core.output_parsers import StrOutputParser

In [118]:
#note retriever se mile token ko we need to merge togther then passes to llm aise alag alg list nhi join krkke bhejne
def format_document(docs):
  return "\n".join(doc.page_content for doc in docs)

In [119]:
#main rag pipeline bana rhe
rag_chain=({"context":retriever | format_document,"question":RunnablePassthrough()}|prompt|llm|StrOutputParser())
#explain phele retriever se mile token ko we need to merge togther then passes to llm aise alag alg list nhi join krkke bhejne  so retriever ko join krne ke liye join_document mien bhejo fir puri input ko prompt mien dallo fir prompt ko llm mien and llm se mile ans ko convert in str for req answer..

In [120]:
rag_chain.invoke("is raat akeli hai presnt on prime?")


' Yes, "Raat Akeli Hai" is available to watch on Prime Video. You can stream movies, TV shows, sports, and live TV, including "Raat Akeli Hai," on Prime Video. Please note that availability may vary depending on your location.'